In [24]:
from __future__ import print_function

import keras
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Input, Merge, ZeroPadding2D, merge, add
from keras.models import load_model
from keras import optimizers
from keras import losses
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

import os, glob, sys, threading
import scipy.io
from scipy import ndimage, misc
import numpy as np
import re
import math
import os
import cv2
import numpy as np
import re
import time

In [25]:
HARD_RESET = False

In [26]:
from random import shuffle
from glob import glob

if HARD_RESET:
    files = glob("data//Scraped//train64_LR//*")
    shuffle(files)

    files = np.array(files)
    splitter = lambda x : x.split('\\')[1]
    filenames = np.vectorize(splitter)(files)
    
    np.save('train_filenames_mapping.npy', filenames)

In [27]:
filenames = np.load('train_filenames_mapping.npy')

In [28]:
low_image_dir = 'data//Scraped//train64_LR'
high_image_dir = 'data//Scraped//train64_HR'

limit_count = 200000

if HARD_RESET:
    data = []
    label = []

    count = 0

    iterator_dir = os.listdir(low_image_dir)

    for filename in filenames:
        count += 1
        if filename.endswith(".png"):
            l_image_filename = os.path.join(low_image_dir, filename)
            h_image_filename = os.path.join(high_image_dir, filename)
            
            im = cv2.imread(l_image_filename)
            if im is None:
                count -= 1
                continue
            a = np.asarray(im)

            im2 = cv2.imread(h_image_filename)
            if im2 is None:
                count -= 1
                continue
            a2 = np.asarray(im2)

            data.append(a)
            label.append(a2)
        else:
            pass

        if count >= limit_count:
            break

    data = np.array(data)
    label = np.array(label)
    
    np.save("data-{}.npy".format(limit_count), data)
    np.save("label-{}.npy".format(limit_count), label)

In [29]:
data = np.load("data-{}.npy".format(limit_count))
label = np.load("label-{}.npy".format(limit_count))

In [13]:
LIM_SAMPLE = len(data)

data = data[:LIM_SAMPLE]
label = label[:LIM_SAMPLE]

In [30]:
OFFSET = 500000
validation_count = 1000

if HARD_RESET:
    data_val = []
    label_val = []

    count = 0

    for filename in filenames[OFFSET:]:
        count += 1
        if filename.endswith(".png"): 
            l_image_filename = os.path.join(low_image_dir, filename)
            h_image_filename = os.path.join(high_image_dir, filename)
            
            im = cv2.imread(l_image_filename)
            if im is None:
                count -= 1
                continue
            a = np.asarray(im)

            im2 = cv2.imread(h_image_filename)
            if im2 is None:
                count -= 1
                continue
            a2 = np.asarray(im2)

            data_val.append(a)
            label_val.append(a2)
        else:
            pass

        if count >= validation_count:
            break

    data_val = np.array(data_val)
    label_val = np.array(label_val)
    
    np.save("data-val-{}.npy".format(validation_count), data_val)
    np.save("label-val-{}.npy".format(validation_count), label_val)

In [31]:
data_val = np.load("data-val-{}.npy".format(validation_count))
label_val = np.load("label-val-{}.npy".format(validation_count))

In [34]:
from keras.utils.vis_utils import plot_model

IMG_SIZE = (64, 64, 3)
BATCH_SIZE = 64
EPOCHS = 100
MODEL_NAME = "VDSR (18 layer CNN)"

input_img = Input(shape=IMG_SIZE)

last_layer = input_img
for i in range(18):
    model = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(last_layer)
    last_layer = Activation('relu')(model)

output = Conv2D(3, (3, 3), padding='same', kernel_initializer='he_normal')(last_layer)
output_img = add([output, input_img])

model = Model(input_img, output_img)

model.load_weights('checkpoints\\tmp.hdf5')

adam = Adam(lr=0.00005)
model.compile(adam, loss='mse', metrics=["mse"])

model.summary()

plot_model(model, show_shapes=True, to_file='VDSR.png')

filepath="./checkpoints/" + MODEL_NAME + "-weights-improvement-{epoch:02d}-{mean_squared_error:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="mse", verbose=1, mode='max')
callbacks_list = [checkpoint]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 64, 64, 64)   1792        input_3[0][0]                    
__________________________________________________________________________________________________
activation_37 (Activation)      (None, 64, 64, 64)   0           conv2d_39[0][0]                  
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 64, 64, 64)   36928       activation_37[0][0]              
__________________________________________________________________________________________________
activation

In [14]:
model.fit(
        data, label, 
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=2,
        callbacks=callbacks_list,
        validation_data=(data_val, label_val)
)

Train on 200000 samples, validate on 1000 samples
Epoch 1/100
 - 1151s - loss: 5.5850 - mean_squared_error: 5.5850 - val_loss: 7.1318 - val_mean_squared_error: 7.1318

Epoch 00001: saving model to ./checkpoints/VDSR (18 layer CNN)-weights-improvement-01-5.58.hdf5
Epoch 2/100
 - 1133s - loss: 5.5774 - mean_squared_error: 5.5774 - val_loss: 7.1354 - val_mean_squared_error: 7.1354

Epoch 00002: saving model to ./checkpoints/VDSR (18 layer CNN)-weights-improvement-02-5.58.hdf5
Epoch 3/100
 - 1133s - loss: 5.5711 - mean_squared_error: 5.5711 - val_loss: 7.1410 - val_mean_squared_error: 7.1410

Epoch 00003: saving model to ./checkpoints/VDSR (18 layer CNN)-weights-improvement-03-5.57.hdf5
Epoch 4/100
 - 1133s - loss: 5.5653 - mean_squared_error: 5.5653 - val_loss: 7.1115 - val_mean_squared_error: 7.1115

Epoch 00004: saving model to ./checkpoints/VDSR (18 layer CNN)-weights-improvement-04-5.57.hdf5
Epoch 5/100
 - 1134s - loss: 5.5592 - mean_squared_error: 5.5592 - val_loss: 7.1181 - val_mean

SystemError: <built-in function Trainer_train_minibatch> returned a result with an error set